In [1]:
pims.__version__

NameError: name 'pims' is not defined

In [4]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
from flowdec.nb import utils as nbutils 
from flowdec import data as fd_data
import pims
from pims import ND2_Reader
# import pims
from flowdec import restoration as fd_restoration
from flowdec import data as fd_data
from flowdec import psf as fd_psf
import dask
import dask.array as da
import tifffile as tf
# from nd2reader import ND2Reader
# from pims import ND2_Reader as ND2Reader
# import aicsimageio.vendor.omexml as ome
import glob
import time, sys
sys.path.insert(0,'./libraries/')
from deco_libraries import update_progress, pims_nd2meta2OMEXML, observer, init_RL_algo, deconv, depth_divide

In [5]:
dirname = "/home/jmamede/imscope/JM/2020/20200626/igfp1_caruby5_continue001trigger002.nd2"

In [7]:
#THINGS TO CHANGE!!!!! Always keep /*.nd2 at the end
#For gui make a function with all this input for verification purposes when loading the files
# dirname = "/run/user/1000/gvfs/smb-share:server=rup-isilon100.rush.edu,share=user/J/jmamede/BigData/Stephanie/M15d_iGFPCAru3_01062021_PBN_VitC_Rutin_NH4Cl_nuc_Vera_pump.nd2"
# dirname = "D:/Stephanie/20210408MDM/*.nd2"
# dirname = "D:/JM/20201222caru3/Carubyigfp_F2_Nh4cl_vitc_rutin_BVD002.nd2"


filelist = glob.glob(dirname)
filelist.sort()
# filelist.pop(1)

print(filelist)
print(filelist[0])
#Load the first file for reference and what's inside the file
frames =  pims.open(filelist[0])
print(frames.sizes)
#Grab metadata
metadata = frames.metadata
# print(metadata['channels'])

#If you don't have enough VRAM divide the XY to do the deconvolution that is automatically assembled after
xdivide = 2
ydivide = 2
depthdivide = depth_divide(xdivide, ydivide)

['/home/jmamede/imscope/JM/2020/20200626/igfp1_caruby5_continue001trigger002.nd2']
/home/jmamede/imscope/JM/2020/20200626/igfp1_caruby5_continue001trigger002.nd2
{'x': 2048, 'y': 2044, 'c': 3, 't': 67, 'm': 31, 'z': 23}


/home/jmamede/anaconda3/lib/python3.7/site-packages/pims/base_frames.py:472: UserWarning: Please call FramesSequenceND.__init__() at the start of thethe reader initialization.
  warn("Please call FramesSequenceND.__init__() at the start of the"


In [ ]:
frames.metadata_text
# test = filelist.pop(0)

In [14]:
#Need to make this a big more automatic, but indexing all the files and iterations is not easy
#In the version where the PSF is estimated by flowdec is easier because everything was set to automatic

PSFdir="/home/jmamede/Rush/PSF/"
# psf647 = tf.imread(PSFdir+"/AF647_0.3_EMCCD.tif")
# psfruby =  tf.imread(PSFdir+'/mRuby3_0.3_EMCCD.tif')
# psfgfp =  tf.imread(PSFdir+'/green_0.3_EMCCD.tif')
# psfdapi =  tf.imread(PSFdir+'/DAPI_0.3_EMCCD.tif')
# psf647 = tf.imread(PSFdir+"/AF647_0.5_EMCCD.tif")
# psfruby =  tf.imread(PSFdir+'/mRuby3_0.5_EMCCD.tif')
# psfgfp =  tf.imread(PSFdir+'/green_0.5_EMCCD.tif')
# psfdapi =  tf.imread(PSFdir+'/DAPI_0.5_EMCCD.tif')
psf647 = tf.imread(PSFdir+"/AF647.tif")
psfruby =  tf.imread(PSFdir+'/mRuby3.tif')
psfgfp =  tf.imread(PSFdir+'/green.tif')
psfdapi =  tf.imread(PSFdir+'/DAPI.tif')
# psfruby = psf647
# psfgfp = psf647
# psfdapi = psf647

#Clip Top and bottom, I found that if the PSF is calculated with distance from coverslip, it is flipped from Fiji
#As in, it is Top to Bottom, I acquire the images Bottom to Top
remove = 8
# remove = 14
psf647 = np.flip(psf647,
#                  axis=0
                )[remove:-remove,64:-64,64:-64]

psfgfp = np.flip(psfgfp,
#                  axis=0
                )[remove:-remove,64:-64,64:-64]
psfdapi = np.flip(psfdapi,
#                   axis=0
                 )[remove:-remove,64:-64,64:-64]
psfruby = np.flip(psfruby,
#                   axis=0
                 )[remove:-remove,64:-64,64:-64]

print(psf647.shape)
print(psfruby.shape)

(19, 128, 128)
(19, 128, 128)


In [15]:
algo =init_RL_algo(psfgfp.ndim
                   ,pad_mode='2357',pad_min=(2,24,24)
#                    ,pad_mode='log2',pad_min=(1,1,1)
                  )
filelist

['/home/jmamede/imscope/JM/2020/20200626/igfp1_caruby5_continue001trigger002.nd2']

In [16]:
# fname2 = filelist[3]
# fname1 = filelist[0]
# # frames.parser._raw_metadata.image_metadata
# def get_deltaT(filename1,filename2):
    
#     import pathlib

#     fname1 = pathlib.Path(filename1)
#     fname2 = pathlib.Path(filename2)
#     import datetime
#     mtime2 = datetime.datetime.fromtimestamp(fname2.stat().st_ctime)
#     mtime1 = datetime.datetime.fromtimestamp(fname1.stat().st_ctime)
# #     print(mtime2,mtime1)
# #     print(fname2.stat().st_ctime - fname1.stat().st_ctime)
#     return fname2.stat().st_ctime - fname1.stat().st_ctime
#     # time_offset = 71146.03463482857
    
# print(get_deltaT(fname1,fname2)/60)

# time_offset = get_deltaT(fname1,fname2)

In [ ]:
savePRJ = True
saveStack = False
# time_offset = 0
# maxT = 25
maxT = None
concattime = False
for fname in filelist:
    #to add the real time of experiment when we have to split it into different files
    if concattime:
#         time_offset = get_deltaT(filelist[0],fname)
        time_offset = get_deltaT(fname1,fname)
    else: time_offset = 0
    frames =  pims.open(fname)
    frames.iter_axes = 't'  # 't' is the default already
    frames.bundle_axes = 'zyx'  # when 'z' is available, this will be default
    
    channels = [frames.metadata['plane_0']['name'],frames.metadata['plane_1']['name'],frames.metadata['plane_2']['name']]
#     channels = frames.metadata['channels']

    metadata = frames.metadata
#     if saveStack:
#         xml = pims_nd2meta2OMEXML(frames,time_offset=time_offset, maxT=maxT)
#     if savePRJ:
#         prjxml = pims_nd2meta2OMEXML(frames, project=True,time_offset=time_offset, maxT=maxT)

    chunk_size=(frames.sizes['z'],frames.sizes['y']//ydivide,frames.sizes['x']//xdivide)

    # frames.iter_axes = 'v'
    # for frame in frames:
    for visit in range(frames.sizes['m']):
#     for visit in range(21,31):

        frames.default_coords['m'] = visit
        stackfile = fname[:-4]+"_v"+str(visit+1).zfill(2)+'.tiff'
        prjfile = fname[:-4]+"_v"+str(visit+1).zfill(2)+'_PRJ.tiff'
        
        if saveStack:
            tifstack = tf.TiffWriter(stackfile, bigtiff=False,imagej=False)
            firststk = True
        if savePRJ:
            tifprj = tf.TiffWriter(prjfile, bigtiff=False, imagej=False)
            first = True
            
        if maxT == None:
            counterT = frames.sizes['t']
        else: counterT = maxT
        for time in range(counterT):
#         for time in range(25):

            res = np.zeros(
                (frames.sizes['z'],frames.sizes['c']
                 ,frames.sizes['y'],frames.sizes['x'])
                , dtype=np.float32)
            for i in range(frames.sizes['c']):
                frames.default_coords['c'] = i
#                     print('Channel:',metadata['channels'][i])
                ch = frames.metadata['plane_'+str(i)]['name']
                # ['a647', 'DAPI', 'GFP']
                # ['FITC', 'mRuby3', 'miRFP670']
                # ['FRET-gYFP-dsRED', 'mRuby', 'GFP', 'DAPI']
                # ['a647', 'DAPI', 'GFP']
                if ch == 'DAPI' or ch  == '470 nm':
                    psf = psfdapi
                elif ch == 'miRFP670' or ch == 'a647' or ch == 'mirfp670' or ch == 'farRED-EM':
                    psf = psf647
                elif ch == 'GFP' or ch == 'FITC'  or ch == 'GREEN-EM' or ch == 'FITC (Em)':
                    psf = psfgfp
                elif ch == 'mRuby3' or ch == 'mRuby' or ch ==  '555 nm' or ch == 'RED-EM':
                    psf = psfruby
                elif ch == 'FRET-gYFP-dsRED':
                    psf = psfruby
                else: psf = psfruby

                arr = da.from_array(frames[time]           
                                    , chunks=chunk_size)


                res[:,i,:,:] = da.map_overlap(deconv,arr, depth  = depthdivide ,boundary='reflect',
                                    dtype='float32', algo=algo,
                                    psf=psf, iters=100,
                                    use_ram=True,
                                            
                                             ).compute(num_workers=1)

                # If we need to swap Channel and Z in the future
                #             img5d = np.swapaxes(img5d,0,1)

                update_progress( (visit+1) * (time+1) / (frames.sizes['m']* counterT) )
                print("Visit Point: {}/{} \t Time:{}/{} \t Channel:{}/{}".format(
                        visit+1, frames.sizes['m']
                        ,time+1,  counterT
                        , i+1, frames.sizes['c']))
# Make Function
            if saveStack:

                if firststk:
                    tifstack.save(res.astype(np.uint16),
                        compress='ZLIB',
#                         description = xml.to_xml(),
                        photometric='minisblack',
                        metadata= None,
                        contiguous=False
                        )
                    firststk = False
                    print('blup')
                else:
                    tifstack.save(res.astype(np.uint16),
                        compress='ZLIB',
#                         description = xml.to_xml(),
#                         datetime= True,
                        photometric='minisblack',
                        metadata= None,
                        contiguous=False
                        )
            if savePRJ:
                if first:
                    tifprj.save(res.astype(np.uint16).max(axis=0),
                        compress='ZLIB',
#                         description = prjxml.to_xml(),
                        photometric='minisblack',
                        #, datetime= True
                        metadata= None,
                        contiguous=False
                        )
                    first = False
                else:
                    
                    tifprj.save(res.astype(np.uint16).max(axis=0),
                        compress='ZLIB',
#                         description = xml.to_xml(),
                        photometric='minisblack',
                        #, datetime= True
                        metadata= None,
                        contiguous=False,
                        )
        if saveStack:
            tifstack.close()
        if savePRJ:
            tifprj.close()

    update_progress(1)

Progress: [--------------------] 0.3%
Visit Point: 1/31 	 Time:6/67 	 Channel:1/3


In [18]:
print('done')
from numba import cuda
cuda.select_device(0)
cuda.close()
#the memory was released here!
cuda.select_device(0)

done


<weakproxy at 0x0000027EF3C12C78 to Device at 0x0000027EF24E52C8>